In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EPNXLQD9E to authenticate.


Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-241085
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-241085


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster-udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster in the provided workspace, so we use it.')
except ComputeTargetException:
    comp_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=3)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, comp_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
    "--C" : uniform(0.0,10.0),
    "--max_iter" : choice(10,20,30,40,50,60,70,80,90,100)
    }
)

# Specify a Policy ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, 
                      evaluation_interval = 2,
                      delay_evaluation = 10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job

src = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=cluster_name,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy. ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs = 4,
                             max_concurrent_runs = 2)



In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config) 

#show run details with the widget.
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [18]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")

print(best_run)

best_run_params = best_run.get_details()['runDefinition']['parameters']
best_run_metrics = best_run.get_metrics()

print("Information about the best model: \n")
print("Run id: {}".format(best_run.id))
print("Accuracy: {}".format(best_run_metrics['Accuracy']))
print("Hyperparameter values: \n")
print(dict({best_run_params[1::2][0]['name'][9:]: best_run_params[1::2][0]['value']}))
print(dict({best_run_params[1::2][1]['name'][9:]: best_run_params[1::2][1]['value']}))

model_name = "HyperDrive_model.pkl"
file_name = "outputs/" + model_name

#if not os.path.exists('outputs/'):
#    os.makedirs('outputs/')
#joblib.dump(value=best_run, filename=file_name)
description = 'HyperDrive best forescasting model'
model = best_run.register_model(model_name='HyperDriveModel', description=description, model_path=file_name)
print(model)

Run(Experiment: udacity-project,
Id: HD_4bc2f306-294d-4f92-aef4-439a9d356251_0,
Type: azureml.scriptrun,
Status: Completed)
Information about the best model: 

Run id: HD_4bc2f306-294d-4f92-aef4-439a9d356251_0
Accuracy: 0.9108883575501195
Hyperparameter values: 

{'C': '3.082505500069325'}
{'max_iter': '60'}
Model(workspace=Workspace.create(name='quick-starts-ws-241085', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-241085'), name=HyperDriveModel, id=HyperDriveModel:2, version=2, tags={}, properties={})


In [19]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
initial_data = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [20]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(initial_data)



In [21]:
# From the distrbution of the target variable, we can see that we are facing an unbalanced project
y.value_counts()

0    29258
1     3692
Name: y, dtype: int64

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset, Datastore

# Split data into train and test sets. Set random state to ensure that we have the same partitions for HyperDrive and AutoML
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

# Register the train and test sets
datastore_ws = Datastore.get(ws, 'workspaceblobstore')
train_ds = Dataset.Tabular.register_pandas_dataframe(train_df, datastore_ws, "train data", show_progress=True)
test_ds = Dataset.Tabular.register_pandas_dataframe(test_df, datastore_ws, "test data", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/7faf7911-314b-4d49-93cd-b2e02ac70dd1/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [29]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    blocked_models = ["LogisticRegression"],
    primary_metric = "accuracy",
    training_data=train_ds,
    validation_data=test_ds,
    label_column_name='y',
    compute_target=cpu_cluster)

In [30]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cluster-udacity with default configuration
Running on remote compute: cluster-udacity


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5497413d-ad60-41a2-ba49-2f6f41770141,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in th

In [34]:
automl_best_run = automl_run.get_best_child()
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5497413d-ad60-41a2-ba49-2f6f41770141_35,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [39]:
print("ID of the best model: ",automl_best_run.id)

ID of the best model:  AutoML_5497413d-ad60-41a2-ba49-2f6f41770141_35


In [55]:
# Retrieve and save your best automl model.
_, automl_best_model = automl_run.get_output()
print(automl_best_model)

joblib.dump(automl_best_model, 'outputs/AutoML_model.pkl')


Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=2.5, subsample=1, tree_method='auto'))], verbose=False))

['outputs/AutoML_model.pkl']

Current provisioning state of AmlCompute is "Deleting"



In [56]:
cpu_cluster.delete()

In [57]:
# Check status of the cluster for deletion 
cpu_cluster

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-241085', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-241085'), name=cluster-udacity, id=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourceGroups/aml-quickstarts-241085/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-241085/computes/cluster-udacity, type=AmlCompute, provisioning_state=Deleting, location=southcentralus, tags=None)

Current provisioning state of AmlCompute is "Deleting"

